# Download Subdirectories From FTP

#### Updated: Dec 5, 2022

#  

Determine how to download subdirectories from FTP. Record download in a 'download_log'. Move downloaded subdirectories from '01_downloading' folder to '02_downloaded' folder. 

In [1]:
import pandas as pd
import os
import shutil
import time
from datetime import datetime
import ftplib

In [2]:
rootDir_local = 'C:/Users/rnaidoo/Documents/Projects_data/Alouette_I/' #files on C:/ are not persistent on VDI
rootDir_U = rootDir_local #'U:/Data_Science/Projects_data/Alouette_I/'
downloadingDir = rootDir_local + '01_downloading/'
downloadedDir = rootDir_local + '02_downloaded/'
resultDir = rootDir_U + '05_result/'

In [3]:
HOSTNAME = "donnees-data.asc-csa.gc.ca"
USERNAME = "Anonymous"
PASSWORD = ""

# Connect FTP Server
ftp = ftplib.FTP(HOSTNAME, USERNAME, PASSWORD)
print('Connected to ftp server: ' + HOSTNAME)

ftp_rootpath = '/users/OpenData_DonneesOuvertes/pub/AlouetteData/Alouette Data/'

Connected to ftp server: donnees-data.asc-csa.gc.ca


#  

#### Functions:

In [10]:
def move_images(old_dir, new_dir, roll, subdir):
    oldDir = old_dir + roll + '/' + subdir + '/'
    newDir = new_dir + roll + '/' + subdir + '/'
    os.makedirs(newDir, exist_ok=True)
    for file in os.listdir(oldDir):
        os.rename(oldDir+file, newDir+file)
    shutil.rmtree(old_dir + roll + '/')

#  

#### Download a subdirectory by FTP:

In [4]:
roll = 'R014207815'
subdirectory = '3488-15A'

In [5]:
saveDir = downloadingDir + roll + '/' + subdirectory + '/'
os.makedirs(saveDir, exist_ok=True)
ftp.cwd(ftp_rootpath + '/' + roll + '/' + subdirectory + '/')

'250 Directory successfully changed.'

In [6]:
start = time.time()
n_dl = len(ftp.nlst())
print('Downloading ' + roll + '/' + subdirectory + '/ subdirectory ('  + str(n_dl) + ' images)')
for file in ftp.nlst():
    local_file = open(saveDir + file, "wb")
    ftp.retrbinary("RETR " + file, local_file.write)
    local_file.close()
    #print('Downloaded: ' + file)
end = time.time()
t = end - start
print('Download time for subdirectory: ' + str(round(t/60, 1)) + ' min')

Download time for subdirectory: 4.4 min


Download to C:/ (CSA laptop, on VPN) --> ~ 23 min <br>
Download to C:/ (CSA laptop, off VPN) --> 4.5 min <br>
Download to C:/ (VDI HP) --> 1.1 min <-- downloading locally seems to be faster <br>
Download to U:/ (VDI HP) --> 1.3 min

Record subdirectory name in download_log:

In [7]:
df_result = pd.DataFrame({
    'Roll': roll,
    'Subdirectory': subdirectory,
    'Images_downloaded': n_dl,
    'Download_time': t,
    'Download_timestamp': datetime.fromtimestamp(end)
}, index=[0])
df_result

,Roll,Subdirectory,Images_downloaded,Download_time,Download_timestamp
0,R014207815,3488-15A,273,266.427046,2022-12-05 16:19:13.747687


In [8]:
if os.path.exists(resultDir + 'download_log.csv'):
    df_log = pd.read_csv(resultDir + 'download_log.csv')
    df_update = pd.concat([df_log, df_result], axis=0, ignore_index=True)
    df_update.to_csv(resultDir + 'download_log.csv', index=False)
else:
    df_result.to_csv(resultDir + 'download_log.csv', index=False)

Move fully downloaded subdirectory to 02_processing folder:

In [ ]:
move_images(old_dir=downloadingDir, new_dir=downloadedDir, roll=roll, subdir=subdirectory)

#  

#### Determine how many subdirectories there are:

Estimate how long it would take the download all subdirectories. Estimate what would be the total size.

In [11]:
ftp.cwd(ftp_rootpath)
rolls_list = ftp.nlst()

In [12]:
n_subdir = 0
for roll in rolls_list:
    ftp.cwd(roll)
    subdir_list = ftp.nlst()
    n_subdir_ = len(subdir_list)
    print('Roll ' + roll + ' has ' + str(n_subdir_) + ' subdirectories.')
    n_subdir += n_subdir_
    ftp.cwd(ftp_rootpath)

Roll R014207815 has 60 subdirectories.
Roll R014207816 has 60 subdirectories.
Roll R014207821 has 60 subdirectories.
Roll R014207823 has 60 subdirectories.
Roll R014207824 has 60 subdirectories.
Roll R014207832 has 60 subdirectories.
Roll R014207840 has 60 subdirectories.
Roll R014207841 has 60 subdirectories.
Roll R014207842 has 59 subdirectories.
Roll R014207844 has 60 subdirectories.
Roll R014207907F has 61 subdirectories.
Roll R014207908F has 60 subdirectories.
Roll R014207909F has 55 subdirectories.
Roll R014207929F has 57 subdirectories.
Roll R014207930F has 73 subdirectories.
Roll R014207938 has 59 subdirectories.
Roll R014207939 has 59 subdirectories.
Roll R014207940F has 58 subdirectories.
Roll R014207942 has 59 subdirectories.
Roll R014207943 has 60 subdirectories.
Roll R014207946 has 60 subdirectories.
Roll R014207947 has 60 subdirectories.
Roll R014207948 has 60 subdirectories.
Roll R014207949 has 60 subdirectories.
Roll R014207951 has 61 subdirectories.
Roll R014207953 has

In [13]:
print('There are ' + str(n_subdir) + ' subdirectories.')

There are 2638 subdirectories.


If one subdirectory takes ~ 1.1 min to download, it would take ~ 48.4 h to download all images...

If one subdirectory is ~ 336 MB, the total size of all of the images is ~ 889 GB (0.9 TB)...